## Train a Scikit-Learn Model using SageMaker Container Mode
### Bring Your Own Container (BYOC) + SageMaker Async Inference

### 1. Create Train Script 

In [ ]:
%%file train
#!/usr/bin/env python

from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import pickle
import os


np.random.seed(123)

# Define paths for Model Training inside Container.
INPUT_PATH = '/opt/ml/input/data'
OUTPUT_PATH = '/opt/ml/output'
MODEL_PATH = '/opt/ml/model'
PARAM_PATH = '/opt/ml/input/config/hyperparameters.json'

# Training data sitting in S3 will be copied to this location during training when used with File MODE.
TRAIN_DATA_PATH = f'{INPUT_PATH}/train'
TEST_DATA_PATH = f'{INPUT_PATH}/test'

def train():
    print("------- [STARTING TRAINING] -------")
    train_df = pd.read_csv(os.path.join(TRAIN_DATA_PATH, 'train.csv'), names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
    train_df.head()
    X_train = train_df[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
    y_train = train_df['class']
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    # Save the trained Model inside the Container
    with open(os.path.join(MODEL_PATH, 'model.pkl'), 'wb') as out:
        pickle.dump(knn, out)
    print("------- [TRAINING COMPLETE!] -------")
    
    print("------- [STARTING EVALUATION] -------")
    test_df = pd.read_csv(os.path.join(TEST_DATA_PATH, 'test.csv'), names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
    X_test = train_df[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
    y_test = train_df['class']
    acc = knn.score(X_test, y_test)
    print('Accuracy = {:.2f}%'.format(acc * 100))
    print("------- [EVALUATION DONE!] -------")

if __name__ == '__main__':
    train()

### 2. Create Serve Script

In [ ]:
%%file serve
#!/usr/bin/env python

from flask import Flask, Response, request
from io import StringIO
import pandas as pd
import logging
import pickle
import os


app = Flask(__name__)

MODEL_PATH = '/opt/ml/model'

# Singleton Class for holding the Model
class Predictor:
    model = None
    
    @classmethod
    def load_model(cls):
        print('[LOADING MODEL]')
        if cls.model is None:
            with open(os.path.join(MODEL_PATH, 'model.pkl'), 'rb') as file_:
                cls.model = pickle.load(file_)
        print('MODEL LOADED!')
        return cls.model
    
    @classmethod
    def predict(cls, X):
        clf = cls.load_model()
        return clf.predict(X)

@app.route('/ping', methods=['GET'])
def ping():
    print('[HEALTH CHECK]')
    model = Predictor.load_model()
    status = 200
    if model is None:
        status = 404
    return Response(response={"HEALTH CHECK": "OK"}, status=status, mimetype='application/json')

@app.route('/invocations', methods=['POST'])
def invoke():
    data = None

    # Transform Payload in CSV to Pandas DataFrame.
    if request.content_type == 'text/csv':
        data = request.data.decode('utf-8')
        data = StringIO(data)
        data = pd.read_csv(data, header=None)
    else:
        return flask.Response(response='This Predictor only supports CSV data', status=415, mimetype='text/plain')

    logging.info('Invoked with {} records'.format(data.shape[0]))
    
    predictions = Predictor.predict(data)

    # Convert from numpy back to CSV
    out = StringIO()
    pd.DataFrame({'results': predictions}).to_csv(out, header=False, index=False)
    result = out.getvalue()

    return Response(response=result, status=200, mimetype='text/csv')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

### 3. Build a Docker Image and Push to ECR

<p>Build the docker image and push to ECR and have the image URI handy for the next steps.</p>

In [ ]:
!docker build -t sagemaker-byoc-sklearn -f Dockerfile .

In [ ]:
%%sh

# Specify a name to your custom container
container_name=sagemaker-byoc-sklearn
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

### 4. Train your Custom Sklearn Model using SageMaker Training

### Imports 

In [ ]:
from sagemaker.serializers import CSVSerializer
import pandas as pd
import sagemaker

### Essentials

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
image_name = 'sagemaker-byoc-sklearn'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'

In [ ]:
image_uri

### Train (using SageMaker)

In [ ]:
WORK_DIRECTORY = '.././DATA'

train_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/train', key_prefix='byoc-sklearn/train')
test_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/test', key_prefix='byoc-sklearn/test')

In [ ]:
train_data_s3_pointer

In [ ]:
test_data_s3_pointer

In [ ]:
model = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    sagemaker_session=session  # ensure the session is set to session
)

In [ ]:
model.fit({'train': train_data_s3_pointer, 'test': test_data_s3_pointer})

In [ ]:
model._current_job_name

### Imports for Inference

In [2]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3
import time

In [3]:
TRAINING_JOB_NAME = 'sagemaker-byoc-sklearn-2022-08-24-15-03-56-118' # Copy this from the AWS SageMaker console
#TRAINING_JOB_NAME = model._current_job_name

In [4]:
sagemaker_session = sagemaker.session.Session()

In [5]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
MODEL_NAME = f'clf-byoc-model-{current_timestamp}'

In [6]:
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

In [7]:
s3_bucket = sagemaker_session.default_bucket()
s3_bucket

'sagemaker-us-east-1-119174016168'

In [8]:
bucket_prefix = 'async_test'

In [10]:
sagemaker_session.create_model_from_job(training_job_name=TRAINING_JOB_NAME, 
                                                     name=MODEL_NAME)

Using already existing model: clf-byoc-model-2022-10-04-23-02-32


'clf-byoc-model-2022-10-04-23-02-32'

### Create Async Endpoint Configuration

In [11]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = f"async-ep-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print(endpoint_config_name)


create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 2 # Number of instances to launch initially.
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": f"s3://{s3_bucket}/{bucket_prefix}/output",
            # (Optional) specify Amazon SNS topics
            "NotificationConfig": {
                "SuccessTopic": "arn:aws:sns:us-east-1:119174016168:success-topic",
                "ErrorTopic": "arn:aws:sns:us-east-1:119174016168:error-topic",
            }
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker will choose an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

async-ep-2022-10-04-23-04-28
Created EndpointConfig: arn:aws:sagemaker:us-east-1:119174016168:endpoint-config/async-ep-2022-10-04-23-04-28


### Create Async Endpoint

In [12]:
# The name of the endpoint.The name must be unique within an AWS Region in your AWS account.
endpoint_name = f"async-ep-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}" 

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [13]:
endpoint_name

'async-ep-2022-10-04-23-04-37'

In [14]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'async-ep-2022-10-04-23-04-37',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint/async-ep-2022-10-04-23-04-37',
 'EndpointConfigName': 'async-ep-2022-10-04-23-04-28',
 'ProductionVariants': [{'VariantName': 'variant1',
   'DeployedImages': [{'SpecifiedImage': '119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker-byoc-sklearn:latest',
     'ResolvedImage': '119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker-byoc-sklearn@sha256:8f57830837b381684f664c661c8532968df02cfc6e2cf169a715b993170d1e5f',
     'ResolutionTime': datetime.datetime(2022, 10, 4, 23, 4, 38, 839000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 2,
   'DesiredInstanceCount': 2}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 10, 4, 23, 4, 38, 230000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 10, 4, 23, 6, 16, 719000, tzinfo=tzlocal()),
 'AsyncInferenceConfig': {'ClientConfig': {'MaxC

### Invoke Async Endpoint

In [15]:
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

In [16]:
input_location = f"s3://{s3_bucket}/async-test/test.csv"

# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)
response

{'ResponseMetadata': {'RequestId': '74bdd4e8-5c7e-43e9-a3d0-98ebde7e3595',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '74bdd4e8-5c7e-43e9-a3d0-98ebde7e3595',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/09c3fbf1-3e6c-491a-968c-2e68e83ab977.out',
   'date': 'Tue, 04 Oct 2022 23:07:21 GMT',
   'content-type': 'application/json',
   'content-length': '54'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/09c3fbf1-3e6c-491a-968c-2e68e83ab977.out',
 'InferenceId': '5626c9ba-ee31-4974-bf32-81cb2e1cf5f1'}

### Invoke Async Endpoint (Exception Scenario)

In [17]:
# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

# Specify the location of the input. Here, a single SVM sample
input_location = f"s3://{s3_bucket}/async-test/bad_test.csv"  # 5 col value is string


# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)
response

{'ResponseMetadata': {'RequestId': 'ae9bc55d-75b7-4627-93c5-b084c6f70af1',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': 'ae9bc55d-75b7-4627-93c5-b084c6f70af1',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/033f0e65-f4a7-48af-8d0f-3c7d18c60bf0.out',
   'date': 'Tue, 04 Oct 2022 23:09:18 GMT',
   'content-type': 'application/json',
   'content-length': '54'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/033f0e65-f4a7-48af-8d0f-3c7d18c60bf0.out',
 'InferenceId': '34ef04f5-2e3f-4d72-868d-35efd1098738'}